**Задание**

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)


In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 1.3 MB 46.1 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 596 kB 65.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [3]:
import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [4]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
test_records = read_gazeta_records("gazeta_test.txt")

In [7]:
HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [11]:
def summurize(rec):

  article_text = ("{}".format(test_records[rec]["text"]))

  input_ids = tokenizer(
      [HANDLER(article_text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
  )["input_ids"]

  output_ids = model.generate(
      input_ids=input_ids,
      max_length=84,
      no_repeat_ngram_size=2,
      num_beams=4
  )[0]

  summary = tokenizer.decode(
      output_ids,
      skip_special_tokens=True,
      clean_up_tokenization_spaces=False
  )

  print('TEXT:   ', article_text)
  print('SUMMARY:', summary)

Проверим результат

In [9]:
summurize(5)

TEXT:    Журналист Питер Суппли Бенсон датского издания Berlingske обратил внимание на усиление России в Арктике, а также на головной патрульный корабль ледового класса «Иван Папанин», который он назвал «новым кулаком» страны в регионе. В статье он указал, что новость о спуске боевого корабля на воду в Санкт-Петербурге получила международный резонанс. Автор материала отметил уникальность ледокола, оснащенного не только пушками, но и ракетами, которые могут быть применены для стрельбы и по другим кораблям, и по позициям противника на берегу. Длина » Ивана Папанина », согласно приведенным данным, составляет 114 м, а водоизмещение — 8,5 тыс. тонн. Ледокол оснастят крылатыми ракетами «Калибр», дальность которых может превышать 200 км. Планируется, что в арсенале этих ледоколов будет по восемь «Калибров». Учитывая эти обстоятельства, автор статьи выразил обеспокоенность в связи с тем, что Дания инвестирует в Арктику и Гренландию слишком мало средств и из-за этого отстает от России, которая 

In [10]:
summurize(7)

TEXT:    Самые известные российские пранкеры Владимир «Вован» Кузнецов и Алексей «Лексус» Столяров, на счету которых множество розыгрышей мировых политиков первого эшелона, позвонили члену конгресса США от Демократической партии Максин Уотерс от имени шведской школьницы, экологической активистки Греты Тунберг и ее отца Сванте. Звонок был сделан в рамках нового анимационного шоу пранкеров Stars save the Earth. Россияне рассказали американскому политику, что Грета якобы сейчас участвует в митинге в штате Северная Каролина «в поддержку экологии острова Чунга-Чанга». В этой связи «Тунберги» предложили Уотерс также поддержать экологию острова (несуществующего), чтобы ее речь затем якобы транслировать на мероприятии, передает РИА «Новости». «Я очень рада, что Грета с вами в Северной Каролине, где вы сосредоточены на защите очень важного острова Чунга-Чанга. Самое главное сейчас — убедиться, что остров защищен, что вы даете ему всю поддержку, которую вы можете дать», — заявила конгрессвумен С

In [17]:
summurize(10)

TEXT:    Спрос на бензин в России в 2019 году упал на 1%. Об этом сообщают «Известия», публикуя результаты исследования группы «Петромаркет» и агентства «Аналитика товарных рынков». Такое снижение спроса стало рекордом за последние годы. По мнению экспертов, тенденция объясняется не высокими ценами на бензин, а снижением реальных доходов населения. Автомобиль стал «довольно дорогим удовольствием», утверждают исследователи. Машина требует затрат на штрафы, техобслуживание и содержание. В результате россияне пересаживаются на такси и общественный транспорт. Некоторые автомобилисты переходят на более дешевый газ. При этом почти 50% россиян считают, что цены на бензин слишком высоки, и из-за этого машиной приходится пользоваться реже. Об этом свидетельствуют данные сентябрьского опроса ВЦИОМ. Среди других причин – дороговизна в обслуживании авто, выплата страховки и налогов (16%), попытки сэкономить средства (15%), пробки на дорогах (14%), забота об экологической обстановке (8% россиян) и 

In [15]:
summurize(55)

TEXT:    Народный артист СССР Василий Лановой рассказал в комментарии «Вечерней газете» о своем самочувствии. По данным СМИ, 85-летнему актеру потребовалась скорая медицинская помощь во время спектакля «Последние луны» в Государственном академическом театре имени Евгения Вахтангова. Из-за ухудшения состояния второй акт постановки пришлось отменить. «Почувствовал недомогание, давление повысилось. Слава богу, скорая прибыла вовремя, мне оказали первую помощь», — пояснил Лановой изданию. Отмечается, что медики осмотрели актера прямо на месте, госпитализация ему не потребовалась. «Сейчас уже все хорошо, чувствую себя отлично, готов работать», — заключил Лановой. В беседе с РЕН ТВ артист заверил, что продолжит творческую карьеру. «Когда у меня будет спектакль, тогда я и буду на сцене, мой спектакль, все нормально, не волнуйтесь», — сказал он. Василий Лановой состоит в труппе Государственного академического театра им. Е. Вахтангова с 1957 года. В феврале этого года спектакль «Посвящение Еве»

In [16]:
summurize(100)

TEXT:    Фрегаты Черноморского флота «Адмирал Макаров» и «Адмирал Григорович» с крылатыми ракетами «Калибр-НК» проходят через турецкие проливы Босфор и Дарданеллы. Они следуют из Севастополя в дальнюю морскую зону, где войдут в состав сил постоянной группировки ВМФ в Средиземном море, передает «Интерфакс» со ссылкой на официального представителя ЧФ Алексея Рулева. Он рассказал, что при переходе экипажи обоих кораблей провели учения на полигонах боевой подготовки флота в Черном море. Как напомнил Рулев , третий фрегат ЧФ этой серии — «Адмирал Эссен» — выполняет задачи в Средиземном море уже с декабря 2019 года. Корабли Черноморского флота относятся к новой серии фрегатов проекта 11356Р/М (также используется обозначение 11357). Эти многоцелевые фрегаты-сторожевики дальней морской зоны предназначены для уничтожения надводных кораблей и судов, подводных лодок и наземных объектов противника. Они также используются для дозорной службы, патрулирования и охраны морских коммуникаций. На борту ф

**Предобученная модель справляется с суммаризацией вполне неплохо, хотя и не без неточностей.**